In [1]:
import tensorflow as tf

In [4]:
tf.executing_eagerly()

True

In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#tf.enable_eager_execution()

import numpy as np
import os
import time

In [26]:
text = open("train.txt").read()

In [27]:
text[:1000]

"INTRODUCTION\n\n\nIn these lectures I propose to consider the four principal tragedies of\nShakespeare from a single point of view. Nothing will be said of\nShakespeare's place in the history either of English literature or of\nthe drama in general. No attempt will be made to compare him with other\nwriters. I shall leave untouched, or merely glanced at, questions\nregarding his life and character, the development of his genius and art,\nthe genuineness, sources, texts, inter-relations of his various works.\nEven what may be called, in a restricted sense, the 'poetry' of the four\ntragedies--the beauties of style, diction, versification--I shall pass\nby in silence. Our one object will be what, again in a restricted sense,\nmay be called dramatic appreciation; to increase our understanding and\nenjoyment of these works as dramas; to learn to apprehend the action and\nsome of the personages of each with a somewhat greater truth and\nintensity, so that they may assume in our imagination

In [28]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 169730 characters


In [29]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

78 unique characters


In [31]:
np.array(vocab)

array(['\n', ' ', '!', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
       'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b',
       'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',
       'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'è', 'é'],
      dtype='<U1')

In [32]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [33]:
text_as_int

array([31, 36, 42, ..., 67, 54,  9])

In [37]:
np.unique(text_as_int, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77]),
 array([ 2780, 27304,     8,   436,    98,    98,    10,  2289,   203,
         1101,    16,    45,    33,    15,    11,    15,    10,     6,
           10,    14,    74,   270,    41,   249,   128,   145,    47,
           77,    97,    36,   163,   463,    59,    51,    99,   117,
           56,   116,    44,     2,    87,   267,   297,    10,    21,
           92,    14,    57,    57,   696, 10596,  1613,  4268,  4685,
        17268,  3434,  2225,  7734,  9815,    94,   688,  4924,  3058,
         9060,  9688,  2464,   185,  7772,  9371, 12971,  3371,  1150,
         2346,   273,  2218,    17,     5,     3]))

In [39]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'INTRODUCTION\n' ---- characters mapped to int ---- > [31 36 42 40 37 26 43 25 42 31 37 36  0]


In [43]:
#For each input sequence, the corresponding targets contain the same length of text, 
#except shifted one character to the right

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])


I
N
T
R
O


In [41]:
examples_per_epoch

1697

In [44]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'INTRODUCTION\n\n\nIn these lectures I propose to consider the four principal tragedies of\nShakespeare fr'
"om a single point of view. Nothing will be said of\nShakespeare's place in the history either of Engli"
'sh literature or of\nthe drama in general. No attempt will be made to compare him with other\nwriters. '
'I shall leave untouched, or merely glanced at, questions\nregarding his life and character, the develo'
'pment of his genius and art,\nthe genuineness, sources, texts, inter-relations of his various works.\nE'


In [45]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [46]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'INTRODUCTION\n\n\nIn these lectures I propose to consider the four principal tragedies of\nShakespeare f'
Target data: 'NTRODUCTION\n\n\nIn these lectures I propose to consider the four principal tragedies of\nShakespeare fr'


In [47]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 31 ('I')
  expected output: 36 ('N')
Step    1
  input: 36 ('N')
  expected output: 42 ('T')
Step    2
  input: 42 ('T')
  expected output: 40 ('R')
Step    3
  input: 40 ('R')
  expected output: 37 ('O')
Step    4
  input: 37 ('O')
  expected output: 26 ('D')


In [48]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [49]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [51]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
    return model

In [52]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [53]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 78) # (batch_size, sequence_length, vocab_size)


In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           19968     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 78)            79950     
Total params: 5,346,894
Trainable params: 5,346,894
Non-trainable params: 0
_________________________________________________________________


In [55]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [56]:
sampled_indices

array([63,  9, 54, 52, 30, 44, 53, 27, 24, 73, 61, 22,  5,  5, 16,  2, 34,
       41, 56, 61, 47, 60, 25,  4, 76, 64, 39, 56, 45, 21, 18, 31, 42, 74,
       43, 24, 46, 35, 71, 69, 62, 69, 33, 59, 11, 32, 45, 42, 50, 51, 48,
       67, 39, 45, 66, 52,  7, 15, 28, 43, 56, 69, 41,  5,  0, 27, 43, 70,
       29, 54, 23,  9, 53, 36, 75, 65, 24, 14, 31, 28, 20, 54, 21, 60, 42,
       16, 77, 40, 45, 61, 49, 35, 13,  3, 42, 58, 66, 37,  8, 30])

In [57]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))



Input: 
 '\nmurderous ambition of Richard III. In most cases the tragic error\ninvolves no conscious breach of r'

Next Char Predictions: 
 "n.ecHVdEBxl?))6!LSgl[kC(èoQgW;8ITyUBYMvtmtKj1JWTab]rQWqc,5FUgtS)\nEUuGeA.dNzpB4IF:e;kT6éRWl_M3'TiqO-H"


In [59]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 78)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.358116


In [60]:
model.compile(optimizer='adam', loss=loss)

In [61]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [62]:
EPOCHS=10

In [63]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
26/26 [==============================] - 64s 2s/step - loss: 3.3371
Epoch 2/10
26/26 [==============================] - 63s 2s/step - loss: 2.9214
Epoch 3/10
26/26 [==============================] - 63s 2s/step - loss: 2.6926
Epoch 4/10
26/26 [==============================] - 65s 2s/step - loss: 2.5326
Epoch 5/10
26/26 [==============================] - 61s 2s/step - loss: 2.4386
Epoch 6/10
26/26 [==============================] - 66s 3s/step - loss: 2.3684
Epoch 7/10
26/26 [==============================] - 61s 2s/step - loss: 2.3020
Epoch 8/10
26/26 [==============================] - 122s 5s/step - loss: 2.2308
Epoch 9/10
26/26 [==============================] - 93s 4s/step - loss: 2.1573
Epoch 10/10
26/26 [==============================] - 71s 3s/step - loss: 2.0871


In [64]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [65]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19968     
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 78)             79950     
Total params: 5,346,894
Trainable params: 5,346,894
Non-trainable params: 0
_________________________________________________________________


In [67]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [68]:
print(generate_text(model, start_string=u"INTRODUCTION"))

INTRODUCTIONf anlo', lis opeces nerletined best int auss prostho mescewed butt of whus his wing
hay ghan ag shichere remestion of apo ons whe tored fhio vetreal the gas her not jut bus
esea he ade ris amlye
in suspeare mathelings in the meatr buse is dimpencent, _Kimthis whom surdse_is in a besinge !fonve weme his sugseredins wero finf is which Fees vay were id treang chareare oferty-it leven txemged, of the henlont, on _Mace sim thouscepeins wich reppiees to be docales Imeging, but equede wive and sead, shes hith of e
salest dyen by canderred litony if of Oseare.]  The hing reseas arpecties,, whe ken-pescaders. an of is bestancade cond bitita fairt or fithe, blatien, cantnds trit cailfard, e folly bot ofer fored that as
lower panf thes frout hethe,
theneshing ancensearasevis rad stime, sothat veas popme plefor ndmaitel. 2OF of ussent thiage, ofe the castence someseny ispe wis oul tley on whincedd tove dica. in weresses deared apracrition, in the heriven he craifechorf in parsationag c